In [ ]:
# import packages
import pandas as pd
import numpy as np
from datetime import datetime
import os

import ast
from itertools import chain

In [ ]:
# read the datasets
orginal_data_path = 'original_data/'

# for movies_metadata
movies_metadata = pd.read_csv(orginal_data_path + 'movies_metadata.csv')
keywords = pd.read_csv(orginal_data_path + 'keywords.csv')
credits = pd.read_csv(orginal_data_path + 'credits.csv')

# for rating_small and rating
ratings_small = pd.read_csv(orginal_data_path + 'ratings_small.csv')
ratings = pd.read_csv(orginal_data_path + 'ratings.csv')
links = pd.read_csv(orginal_data_path + 'links.csv')

In [ ]:
# saved data path
cleaned_data_path = 'cleaned_data/'
os.makedirs(cleaned_data_path[:-1], exist_ok=True)

# movies_metadata

In [ ]:
# general data type cleaning
movies_metadata = movies_metadata[movies_metadata['adult'].isin(['True', 'False'])]
movies_metadata['id'] = movies_metadata['id'].astype(int)
movies_metadata['popularity'] = movies_metadata['popularity'].astype(float)
movies_metadata['release_date'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce')

movies_metadata['adult'] = movies_metadata['adult'].map({'True': True, 'False': False})

movies_metadata['budget'] = movies_metadata['budget'].astype(int)
for col in ['budget', 'revenue', 'vote_average', 'vote_count']:
    movies_metadata[col] = movies_metadata[col].replace(0, None)

In [ ]:
# map information in keywords and credits: keywords, cast, crew
movies_metadata['keywords'] = movies_metadata['id'].map(dict(zip(keywords['id'], keywords['keywords'])))
movies_metadata['cast'] = movies_metadata['id'].map(dict(zip(credits['id'], credits['cast'])))
movies_metadata['crew'] = movies_metadata['id'].map(dict(zip(credits['id'], credits['crew'])))

In [ ]:
# read the columns as list and dict, not str
def get_list_dict(original_data):
    try:
        return ast.literal_eval(original_data)
    except:
        return None


for col in ['belongs_to_collection', 'spoken_languages', 'production_companies', 
            'production_countries', 'genres', 'keywords', 'cast', 'crew']:
    movies_metadata[col] = movies_metadata[col].apply(get_list_dict)

In [ ]:
movies_metadata['belongs_to_collection'] = movies_metadata['belongs_to_collection'].apply(lambda x: [x] if isinstance(x, dict) else [])

In [ ]:
# simplify the columns, only add id as the list
def get_keys(original_list, read_key):
    final_list = []
    try:
        for item in original_list:
            final_list.append(item[read_key])
        return final_list
    except:
        return []

col_dict = {
   'belongs_to_collection': ['collection', 'id'],
    'spoken_languages': ['lang_code_list', 'iso_639_1'],
    'production_companies': ['pro_comp_list', 'id'],
    'production_countries': ['pro_coun_list', 'iso_3166_1'],
    'genres': ['genre_list', 'id'],
    'keywords': ['keywords_list', 'id'],
    'cast': ['cast_id', 'id'],
    'crew': ['crew_list', 'credit_id']
}

supporting_df_dict = {}

for key in col_dict:
    col_name, col_list_name, read_key = key, col_dict[key][0], col_dict[key][1]
    
    movies_metadata[col_list_name] = movies_metadata[col_name].apply(lambda x: get_keys(x, read_key))

    df = pd.DataFrame(list(chain.from_iterable([row for row in movies_metadata[col_name] if isinstance(row, list)]))).\
    drop_duplicates().sort_values(read_key).reset_index(drop=True)

    supporting_df_dict[col_name] = df
    
    print(f"Total unique {col_name}: {df.shape[0]}")

In [ ]:
# put supporting as excel file
with pd.ExcelWriter(cleaned_data_path + 'movie_metadata_supporting.xlsx', engine='openpyxl') as writer:

    for sheet_name, df in supporting_df_dict.items():
        df.to_excel(writer, sheet_name = sheet_name, index = False)

In [ ]:
# change collecton to float, not list
movies_metadata['collection'] = movies_metadata['collection'].apply(lambda x: x[0] if x != [] else None)

In [ ]:
# drop unnecessary columns
movies_metadata = movies_metadata.drop(col_dict.keys(), axis = 1)

In [ ]:
# save files as csv
movies_metadata.to_csv(cleaned_data_path + 'movies_cleaned.csv', index=False)

# ratings_small

In [ ]:
# general data type cleaning
ratings_small['date_time'] = pd.to_datetime(ratings_small['timestamp'], unit='s')
links['imdb_id'] = links['imdbId'].apply(lambda x: f"tt{int(x):07d}" if pd.notnull(x) else None)
ratings_small['imdb_id'] = ratings_small['movieId'].map(dict(zip(links['movieId'], links['imdb_id'])))

In [ ]:
used_cols = ['user_id', 'movie_id', 'rating', 'imdb_id', 'date_time']
ratings_small = ratings_small.rename(columns = {'userId': 'user_id', 'movieId': 'movie_id'})[used_cols]

In [ ]:
# save files as csv
ratings_small.to_csv(cleaned_data_path + 'ratings_small_cleaned.csv', index = False)

# ratings

In [ ]:
# general data type cleaning
ratings['date_time'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings['imdb_id'] = ratings['movieId'].map(dict(zip(links['movieId'], links['imdb_id'])))

In [ ]:
ratings = ratings.rename(columns = {'userId': 'user_id', 'movieId': 'movie_id'})[used_cols]

In [ ]:
# save files as csv
ratings.to_csv(cleaned_data_path + 'ratings_cleaned.csv', index = False)